# Importing Dependences

In [ ]:
!pip install pygeohash

import pandas as pd
import time
import requests
from bs4 import BeautifulSoup
import re
import urllib.parse
import pygeohash as pgh

from google.colab import drive
drive.mount('/content/drive/')

url = "/content/drive/MyDrive/Portfolio/Retail Spatial Analysis/"

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


# Scraping Lease Property Commercial

In [ ]:
def find_lat_lon(address):
    safe_string = urllib.parse.quote_plus(address)
    url = "https://www.google.com/maps/place/" + safe_string
    page = requests.get(url, headers=headers)
    soup = str(BeautifulSoup(page.text, 'html.parser'))
    lat = soup[soup.find("window.APP_INITIALIZATION_STATE"):].split(",")[1]
    lon = soup[soup.find("window.APP_INITIALIZATION_STATE"):].split(",")[2]

    return lat, lon

url = "https://www.realcommercial.com.au/for-lease/{location}/retail/?activeSort={sort}&includePropertiesWithin=includesurrounding&maxFloorArea={maxfloor}&minFloorArea={minfloor}&maxSiteArea={maxsite}&minSiteArea={minsite}&page={curr_page}"
url = url.format(location = "sydney-nsw-2000,sydney-cbd-nsw,sydney-olympic-park-nsw-2127,western-sydney-nsw,south-western-sydney-nsw,north-sydney-nsw-2060,sydney",
                 sort = "date-desc",
                 maxfloor = 150,
                 minfloor = 50,
                 maxsite = 150,
                 minsite = 50,
                 curr_page = 1)
#print(url)
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/78.0.3904.108 Safari/537.36",
    "Accept-Encoding": "gzip, deflate", "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8",
    "DNT": "1", "Connection": "close", "Upgrade-Insecure-Requests": "1"}

curr_page = 1
pages = 0
lists = []
i = 0
while pages == 0 or curr_page <= pages:
    time.sleep(3)

    url = url.replace('page={}'.format(curr_page-1), 'page={}'.format(curr_page))

    page = requests.get(url, headers=headers)
    soup = BeautifulSoup(page.text, 'html.parser')
    results = soup.find('span', class_="SearchResultsHeader_availableResults_2ahBB").text
    if results.find("-") >= 0:
        current_results = int(''.join(filter(str.isdigit, results.split("-")[1].split("of")[0])))
    else:
        current_results = int(results.split(" ")[0])
    max_results = int(re.search('of (.*) results', results).group(1))

    if pages == 0:
        pages = (max_results//current_results) + 1
        lists = soup.find_all('div', class_="ListingCard_listingCard_28qiR")
    else:
        lists += soup.find_all('div', class_="ListingCard_listingCard_28qiR")

    print("Pages : {} ({}-{})".format(curr_page, current_results, max_results))
    print("URL: {}".format(url))

    curr_page+=1

list_url = []
list_address = []
list_price = []
list_size = []
list_car = []
for x in lists:
    try:
        list_url.append("https://www.realcommercial.com.au" + x.find("a", href=True)["href"])
        list_address.append(x.find("a", class_="Address_link_1aaSW")["title"])
        list_price.append(x.find("h3", class_="Price_price_1Q20z _utilities_text-truncate_21ch_").text)
        attr = x.find_all("span", class_="ListingAttributes_attribute_2XqW4")
        if len(attr) == 3:
            list_size.append(attr[0].text)
            list_car.append(attr[1].text)
        else:
            if len(attr) > 1:
                attr = attr[0]

            if "m²" in attr.text:
                list_size.append(attr.text)
                list_car.append(None)
            else:
                list_size.append(None)
                list_car.append(attr.text)
    except:
        print("Failed")
        continue

df = pd.DataFrame(data = {"url": list_url,
                          "address": list_address,
                          "price": list_price,
                          "size": list_size,
                          "car": list_car})

df["lon"] = df.apply(lambda x: float(find_lat_lon(x["address"])[0]), axis = 1)
df["lat"] = df.apply(lambda x: float(find_lat_lon(x["address"])[1][:-1]), axis = 1)
df["geohash"] = df.apply(lambda x: pgh.encode(x["lat"], x["lon"], precision = 8), axis = 1)
df.to_csv("list_property_commercial_sydney.csv", index = False, header = True)

df

Pages : 1 (10-41)
URL: https://www.realcommercial.com.au/for-lease/sydney-nsw-2000,sydney-cbd-nsw,sydney-olympic-park-nsw-2127,western-sydney-nsw,south-western-sydney-nsw,north-sydney-nsw-2060,sydney/retail/?activeSort=date-desc&includePropertiesWithin=includesurrounding&maxFloorArea=150&minFloorArea=50&maxSiteArea=150&minSiteArea=50&page=1
Pages : 2 (20-41)
URL: https://www.realcommercial.com.au/for-lease/sydney-nsw-2000,sydney-cbd-nsw,sydney-olympic-park-nsw-2127,western-sydney-nsw,south-western-sydney-nsw,north-sydney-nsw-2060,sydney/retail/?activeSort=date-desc&includePropertiesWithin=includesurrounding&maxFloorArea=150&minFloorArea=50&maxSiteArea=150&minSiteArea=50&page=2
Pages : 3 (30-41)
URL: https://www.realcommercial.com.au/for-lease/sydney-nsw-2000,sydney-cbd-nsw,sydney-olympic-park-nsw-2127,western-sydney-nsw,south-western-sydney-nsw,north-sydney-nsw-2060,sydney/retail/?activeSort=date-desc&includePropertiesWithin=includesurrounding&maxFloorArea=150&minFloorArea=50&maxSiteAr

,url,address,price,size,car,lon,lat,geohash
0,https://www.realcommercial.com.au/for-lease/pr...,"5 Erskine Street Sydney, NSW 2000","$80,000 P.A. excluding GST",103 m²,None,151.202175,-33.866838,r3gx2dwp
1,https://www.realcommercial.com.au/for-lease/pr...,"Shop 2, 258 Macquarie Road Springwood, NSW 2777",Contact Agent,85 m²,None,150.563766,-33.699612,r65061jj
2,https://www.realcommercial.com.au/for-lease/pr...,"Level 1/16 Campbell Street Haymarket, NSW 2000",Contact Agent,100 m²,None,151.206306,-33.879001,r3gx2b90
3,https://www.realcommercial.com.au/for-lease/pr...,"1A/12 - 26 Regent Street Sydney, NSW 2000",POA,60 m²,None,151.202750,-33.884669,r3gx0xqr
4,https://www.realcommercial.com.au/for-lease/pr...,"Shop 2/488 George Street Windsor, NSW 2756",Contact Agent,70 m²,None,150.808364,-33.617102,r650z8pq
5,https://www.realcommercial.com.au/for-lease/pr...,"shop 4, 323 Windsor Street Richmond, NSW 2753","$550 per week, inc GST & Outgoings",90 m²,None,150.746533,-33.594035,r650yk6k
6,https://www.realcommercial.com.au/for-lease/pr...,"Shops 1 & 2 223 Windsor Street Richmond, NSW 2753",Expression of Interest Invited,133 m²,None,-119.809303,44.242365,9rfg9wjd
7,https://www.realcommercial.com.au/for-lease/pr...,"1 / 64 - 70 Main Street Blacktown, NSW 2148","$8,200 per month + GST",120 m²,None,150.909012,-33.770173,r3grfh2d
8,https://www.realcommercial.com.au/for-lease/pr...,"Shop 2a/458-470 High Street Penrith, NSW 2750",Contact Agent,60 m²,None,150.696168,-33.753497,r3gpvpme
9,https://www.realcommercial.com.au/for-lease/pr...,"12A Westfield Place Blacktown, NSW 2148","$2,450 plus gst per month",120 m²,None,150.907987,-33.769872,r3grcuru
